# Pattern Project

###  Processes we will follow :

###  1 - Preprocessing images (text-independent techniques / text to binary separate lines-> Otsu's algo)
##### 2 - Features Extraction. six features (  histogram/ height/ width/ horizontal projection / ........)
###  3 - Classifier ( K-nearest / NNs ).

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from numpy import inf

############### for Piplining and reading images ############

import os
import glob

############### for classification with KNN #################
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

from sklearn import metrics

############## for claculating time ##################
import time


C:\Users\Jammal\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:


def reading_image(imageName):
    
    # 1. Read image as a grayscale image
    
    img = cv2.imread(imageName)

    im_gray = cv2.imread(imageName, cv2.IMREAD_GRAYSCALE)
    
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # convert image to binary image only to be used in feature extration
    (threshBinary, BinaryImage) = cv2.threshold(im_gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    
    # convert image to binary image only to be used in feature extration
    ## (2) threshold
    th, BinaryImageInverted = cv2.threshold(im_gray, 127, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
    
    # returns image , binary inverted image , binary image , gray image 
    return img, BinaryImageInverted, BinaryImage, im_gray

In [3]:


def draw_up_downLines(img, binaryInvertedImage):
    
    ################ steps to separate texts into lines #################

    # 1. read the source
    # 2. threshed
    # 3. find minAreaRect
    # 4. warp by the rotated matrix
    # 5. find and draw upper(upLine) and lower (DownLine) bounds

    ## (3) minAreaRect on the nozeros

    pts = cv2.findNonZero(binaryInvertedImage)
    ret = cv2.minAreaRect(pts)

    (cx,cy), (w,h), ang = ret
    
    if w>h:
        w,h = h,w
        ang += 90

    ## (4) Find rotated matrix, do rotation
    M = cv2.getRotationMatrix2D((cx,cy), ang, 1.0)
    rotated = cv2.warpAffine(binaryInvertedImage, M, (img.shape[1], img.shape[0]))

    ## (5) find and draw the upper and lower boundary of each lines
    hist = cv2.reduce(rotated,1, cv2.REDUCE_AVG).reshape(-1)

    th = 2
    H,W = img.shape[:2]
    uppers = [y for y in range(H-1) if hist[y]<=th and hist[y+1]>th]
    lowers = [y for y in range(H-1) if hist[y]>th and hist[y+1]<=th]

    rotated = cv2.cvtColor(rotated, cv2.COLOR_GRAY2BGR)


    ###  Draw lines of upper area 
    for y in uppers:
        cv2.line(rotated, (0,y), (W, y), (255,0,0), 1)

    ###  Draw lines of lower area

    for y in lowers:
        cv2.line(rotated, (0,y), (W, y), (0,255,0), 1)

    cv2.imwrite("result.jpg", rotated)
    
    # returns Height, Width , upperLines, LowerLines cords.
    return H, W, uppers, lowers

In [4]:

def divide_to_sub_Images(H, W, uppers, lowers):
    
    # array stores pixels
    pixelNumbers = []
    prevPixel1 = 0
    prevPixel2 = 0
    pixel_position1 = 0
    pixel_position2 = 0
    flage1 = True
    flage2 = True
    
    # to filter array of subImages
    uppers1 =[]
    lowers1 = []
    
    # cal features 1 -> 3
    # f1 = Y_top - Y_UpperBaseLine
    # f2 = Y_upperBaseLine - Y_lowerBaseLine
    # f3 = Y_lowerBaseLine - Y_BottomLine
    
    f1 =[]
    f2 = []
    f3 = []
    
    # calc the row with the most blackwhite and its y position
    blackRow = 0
    YBlack = 0
    
    ###  looping through all the image && writing sub images  ###
    for y1,y2 in zip(uppers,lowers):
        #print(y1)
        
        # if diffrence between two y's is less than 10 piexels then (skip it)
        if abs(y1 - y2) < 30:
            continue
        uppers1.append(y1)
        lowers1.append(y2)
        
        
    ##################### looping through each line #########################
    
    # if yprev - ynext >50 then take it as yupperBaseLine
        j = 0 # lower base line counter
        for i in range(y1,y2-1):
            
            # stop condition
            if y1 + j + 5 > y2 - 1:
                break
            ### getting zero pixels 
            #print( W * (5) - cv2.countNonZero(BinaryImage[ y1 +  j : y1 + j + 5, 0 : W ]))

            j = j + 5
            currentPixel =( W * (5) - cv2.countNonZero(BinaryImage[y1 +  j :  y1 + j + 5 , 0 : W ]) ) * 4.5
            
        # Another strategy
            # baseLine1
            if flage1 and currentPixel >= 1000:
                pixel_position1 = y1 + j + 5
                # append to f1  #####################
                f1.append(abs(y1 - pixel_position1))
                #print(flage1)
                flage1 = False

            # baseLine2
            elif not flage1 and flage2 and currentPixel < 1000:
                pixel_position2 = y1 + j + 5
                
                # append to  f2 ,f3 #################
                f2.append(abs(pixel_position1 - pixel_position2))
                f3.append(abs(pixel_position2 - y2))
                #print(flage2)
                flage2 = False
            
            # finding the most black row
            if blackRow < currentPixel:
                blackRow = currentPixel
                YBlack = currentPixel
                
        #print("#################################line################################################")
        
        # if any of the two lines di not detected skip it
        # filtring the not relevant baseLines && missing baseLines
        
        if len(f1) > len(f2):
            f1.pop()
            flage1 = True
            flage2 = True
            continue
            
            
        if len(f1) < len(f2):
            f2.pop()
            flage1 = True
            flage2 = True
            continue
        
        # if the two baseLines Detected draw them else not
        if not flage1 and not flage2:
            
            # drawing baseLine upper and Lower
            cv2.line(BinaryImage, (0,pixel_position1), (W, pixel_position1), (0,255,0), 1)
            cv2.line(BinaryImage, (0,pixel_position2), (W, pixel_position2), (0,255,0), 1)

            # finding distances between black pixels

            ################################################# TODO  ####################################
            sub_images = cv2.imwrite( "sub1%f.jpg"%(y1) ,BinaryImage[ y1:y2 , 0:W ] )
            
        flage1 = True
        flage2 = True
        
    # assign array again after removing not necessary elements
    uppers = uppers1
    lowers = lowers1
    #print(len(f1))
    #print(len(f2))
    #print(len(uppers1))
    #print(len(lowers1))
    return  f1, f2, f3


In [5]:
def calc_features(f1,f2,f3):
    
    # calc features
    
    FV =[]
    # f1 average
    f1_Av = np.average(f1)
    
    FV.append(f1_Av)
    
    # f2 average
    
    f2_Av = np.average(f2)
    
    FV.append(f2_Av)
    
    # f3 average
    
    f3_Av = np.average(f3)
    
    FV.append(f3_Av)
    
    ############### f4 feature ############
    
    # f4 = f1/f2
    
    f4 = f1_Av/f2_Av
    
    FV.append(f4)
    
    # f5 = f1/f3
    
    f5 = f1_Av/f3_Av
    
    FV.append(f5)

    # f6 = f2/f3
    
    f6 = f2_Av/f3_Av
    
    FV.append(f6)
    
    return FV

## Building and training our model with training data

In [6]:
## main Code Area
'''
img, BinaryImageInverted, BinaryImage, im_gray = reading_image("1.jpg")
H, W, upLines, downLines  = draw_up_downLines(img, BinaryImageInverted)
f1,f2,f3 = divide_to_sub_Images(H, W, upLines, downLines)
'''

'''
# feature vector
FV_X=[]
labeledImageY = np.array([1,1,2,2,3,3])
for filename in glob.glob('./images/*.png'):
    #print(filename)
    img, BinaryImageInverted, BinaryImage, im_gray = reading_image(filename)
    # preprocessing module
    H, W, upLines, downLines  = draw_up_downLines(img, BinaryImageInverted)
    # feature extration module
    _,f1,f2,f3 = divide_to_sub_Images(H, W, upLines, downLines)
    fv = calc_features(f1,f2,f3)
    FV_X.append(fv)
    
FV_X_np = np.asarray(FV_X)
print(FV_X_np)

clf = neighbors.KNeighborsClassifier()
clf.fit(FV_X_np, labeledImageY)
print(clf)
y_predic = clf.predict(FV_X_np)
print(FV_X_np[0])
print(y_predic)
'''


# for test case calcuations
# TODO now ############################################
# getting all paths from the data directory
subdirs = [x[0] for x in os.walk('D:\jupiter\Pattern Project\Project Delivery Instructions\data')]  
print(subdirs)

# looping through all the direcories
testcaseNumber = 0
WriterNumber = 1
FV_X_np_testing = None
FV_X = []

####### true predictions ######## ( to measure accuracy )
TruePredictions = 0

#### writting to file results.txt ####
resultsFile = open("results.txt","w+")
timeFile = open("time.txt","w+")

########### labels for images for testing purpose :D ############## .
labeledImageY = np.array([1,1,2,2,3,3])
start = time.time() # calculating start time

# is nan flage filter
isnanflage = False

for subdir in subdirs:
    # testCase number
       if "0" + str(testcaseNumber) in subdir or testcaseNumber >= 10 :
            #print(str(testcaseNumber))
            #print(WriterNumber)
            # writerNumber
            #print(subdir)
            if  str(testcaseNumber) + "\\" + str(WriterNumber) in subdir :
                
                ################ fetching the images from the directory of each writer #################
                #print("##################writer" + str(WriterNumber) +"######################")
                for filename in glob.glob(subdir + '/*.jpg'):
                    #print(filename)
                    img, BinaryImageInverted, BinaryImage, im_gray = reading_image(filename)
                    # preprocessing module
                    H, W, upLines, downLines  = draw_up_downLines(img, BinaryImageInverted)
                    # feature extration module
                    f1,f2,f3 = divide_to_sub_Images(H, W, upLines, downLines)
                    fv = calc_features(f1,f2,f3)
                    FV_X.append(fv)
                # increament the writer number after each iteration
                WriterNumber = WriterNumber + 1

                    ############ KNN Classifier ################
                if WriterNumber == 4:
                    
                    FV_X_np = np.asarray(FV_X)
                    clf = neighbors.KNeighborsClassifier()
                    #print(FV_X_np)
                    #print(labeledImageY)
                    # converting nan and inf to 0
                    FV_X_np = np.nan_to_num(FV_X_np)
                    FV_X_np[FV_X_np == -inf] = 0
                    FV_X_np[FV_X_np == inf] = 0
                    print(np.round(FV_X_np,2))

                    ########################
                    clf.fit(FV_X_np, labeledImageY)
                    #print(clf)
                    #print(FV_X_np_testing)
                    
                    # converting nan and inf to 0
                    FV_X_np_testing = np.nan_to_num(FV_X_np_testing)
                    FV_X_np_testing[FV_X_np_testing == -inf] = 0
                    FV_X_np_testing[FV_X_np_testing == inf] = 0
                    ########################                    
                    
                    y_predic = clf.predict(FV_X_np_testing.reshape(-1,6))
                    result = y_predic.squeeze()
                    print(result)
                    if result == 1:
                        TruePredictions = TruePredictions + 1
                    # write the file results
                    resultsFile.write(str(result) + "\n")
                    end = time.time() # calculating end time
                    TestCaseTime = end - start
                    timeFile.write(str(round(TestCaseTime* 100) ) + "\n")
                    start = time.time()
                    ############################################
            else:
                # extract features from testing image
                testingFile = subdir + "\\test.jpg"
                #print(filename)
                img, BinaryImageInverted, BinaryImage, im_gray = reading_image(testingFile)
                # preprocessing module
                H, W, upLines, downLines  = draw_up_downLines(img, BinaryImageInverted)
                # feature extration module
                f1,f2,f3 = divide_to_sub_Images(H, W, upLines, downLines)
                fvTesting = calc_features(f1,f2,f3)
                FV_X_np_testing = np.asarray(fvTesting)
                    
            if WriterNumber > 3:
                testcaseNumber = testcaseNumber + 1
                WriterNumber = 1
                FV_X= []
                FV_X_np_testing = None

accuracy = (TruePredictions / 88) *100
print("Accuracy = " + str(accuracy) + " %")
resultsFile.close()
timeFile.close()


['D:\\jupiter\\Pattern Project\\Project Delivery Instructions\\data', 'D:\\jupiter\\Pattern Project\\Project Delivery Instructions\\data\\00', 'D:\\jupiter\\Pattern Project\\Project Delivery Instructions\\data\\00\\1', 'D:\\jupiter\\Pattern Project\\Project Delivery Instructions\\data\\00\\2', 'D:\\jupiter\\Pattern Project\\Project Delivery Instructions\\data\\00\\3', 'D:\\jupiter\\Pattern Project\\Project Delivery Instructions\\data\\01', 'D:\\jupiter\\Pattern Project\\Project Delivery Instructions\\data\\01\\1', 'D:\\jupiter\\Pattern Project\\Project Delivery Instructions\\data\\01\\2', 'D:\\jupiter\\Pattern Project\\Project Delivery Instructions\\data\\01\\3', 'D:\\jupiter\\Pattern Project\\Project Delivery Instructions\\data\\02', 'D:\\jupiter\\Pattern Project\\Project Delivery Instructions\\data\\02\\1', 'D:\\jupiter\\Pattern Project\\Project Delivery Instructions\\data\\02\\2', 'D:\\jupiter\\Pattern Project\\Project Delivery Instructions\\data\\02\\3', 'D:\\jupiter\\Pattern Proje

[[1.4000e+01 4.0000e+01 1.6000e+00 3.5000e-01 8.7500e+00 2.5000e+01]
 [1.7140e+01 4.4290e+01 5.2900e+00 3.9000e-01 3.2400e+00 8.3800e+00]
 [1.8000e+01 3.6000e+01 6.6000e+00 5.0000e-01 2.7300e+00 5.4500e+00]
 [1.5420e+01 4.0000e+01 9.8300e+00 3.9000e-01 1.5700e+00 4.0700e+00]
 [3.0000e+01 5.1670e+01 3.1033e+02 5.8000e-01 1.0000e-01 1.7000e-01]
 [1.4620e+01 7.1920e+01 6.0000e+00 2.0000e-01 2.4400e+00 1.1990e+01]]
1
[[11.   41.    2.4   0.27  4.58 17.08]
 [12.   39.   14.    0.31  0.86  2.79]
 [17.14 44.29  5.29  0.39  3.24  8.38]
 [15.   37.5  23.5   0.4   0.64  1.6 ]
 [19.38 50.62 54.12  0.38  0.36  0.94]
 [21.67 30.   39.17  0.72  0.55  0.77]]
2
[[1.4000e+01 4.0000e+01 1.6000e+00 3.5000e-01 8.7500e+00 2.5000e+01]
 [1.5000e+01 3.7500e+01 2.3500e+01 4.0000e-01 6.4000e-01 1.6000e+00]
 [2.4380e+01 3.3750e+01 4.8800e+00 7.2000e-01 5.0000e+00 6.9200e+00]
 [1.1670e+01 3.5000e+01 2.6700e+00 3.3000e-01 4.3800e+00 1.3120e+01]
 [3.0000e+01 5.1670e+01 3.1033e+02 5.8000e-01 1.0000e-01 1.7000e-01]
 

C:\Users\Jammal\Anaconda3\lib\site-packages\numpy\lib\function_base.py:1128: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\Jammal\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[[12.   39.   14.    0.31  0.86  2.79]
 [11.67 29.17 37.5   0.4   0.31  0.78]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [10.   37.5   4.5   0.27  2.22  8.33]
 [21.67 30.   39.17  0.72  0.55  0.77]]
1
[[ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [14.   40.    1.6   0.35  8.75 25.  ]
 [15.   37.5  23.5   0.4   0.64  1.6 ]
 [10.   37.5   4.5   0.27  2.22  8.33]
 [21.67 30.   39.17  0.72  0.55  0.77]]
2
[[1.1000e+01 4.1000e+01 2.4000e+00 2.7000e-01 4.5800e+00 1.7080e+01]
 [1.1670e+01 2.9170e+01 3.7500e+01 4.0000e-01 3.1000e-01 7.8000e-01]
 [3.0000e+01 5.8330e+01 1.8633e+02 5.1000e-01 1.6000e-01 3.1000e-01]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [2.4380e+01 3.3750e+01 4.8800e+00 7.2000e-01 5.0000e+00 6.9200e+00]
 [1.1670e+01 3.5000e+01 2.6700e+00 3.3000e-01 4.3800e+00 1.3120e+01]]
1
[[3.0000e+01 5.8330e+01 1.8633e+02 5.1000e-01 1.6000e-01 3.1000e-01]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.000

[[1.625e+01 6.375e+01 1.450e+01 2.500e-01 1.120e+00 4.400e+00]
 [5.000e+01 5.500e+01 1.215e+03 9.100e-01 4.000e-02 5.000e-02]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [1.000e+01 3.750e+01 4.500e+00 2.700e-01 2.220e+00 8.330e+00]
 [2.167e+01 3.000e+01 3.917e+01 7.200e-01 5.500e-01 7.700e-01]]
2
[[12.   39.   14.    0.31  0.86  2.79]
 [11.67 29.17 37.5   0.4   0.31  0.78]
 [16.67 35.    2.33  0.48  7.14 15.  ]
 [15.42 40.    9.83  0.39  1.57  4.07]
 [10.   37.5   4.5   0.27  2.22  8.33]
 [21.67 30.   39.17  0.72  0.55  0.77]]
1
[[11.   41.    2.4   0.27  4.58 17.08]
 [11.67 29.17 37.5   0.4   0.31  0.78]
 [18.   36.    6.6   0.5   2.73  5.45]
 [16.67 35.    2.33  0.48  7.14 15.  ]
 [10.   37.5   4.5   0.27  2.22  8.33]
 [21.67 30.   39.17  0.72  0.55  0.77]]
1
[[18.   36.    6.6   0.5   2.73  5.45]
 [15.42 40.    9.83  0.39  1.57  4.07]
 [11.   41.    2.4   0.27  4.58 17.08]
 [12.   39.   14.    0.31  0

[[1.400e+01 4.000e+01 1.600e+00 3.500e-01 8.750e+00 2.500e+01]
 [1.714e+01 4.429e+01 5.290e+00 3.900e-01 3.240e+00 8.380e+00]
 [1.625e+01 6.375e+01 1.450e+01 2.500e-01 1.120e+00 4.400e+00]
 [5.000e+01 5.500e+01 1.215e+03 9.100e-01 4.000e-02 5.000e-02]
 [1.938e+01 5.062e+01 5.412e+01 3.800e-01 3.600e-01 9.400e-01]
 [2.167e+01 3.000e+01 3.917e+01 7.200e-01 5.500e-01 7.700e-01]]
1
[[1.0000e+01 3.7500e+01 4.5000e+00 2.7000e-01 2.2200e+00 8.3300e+00]
 [2.1670e+01 3.0000e+01 3.9170e+01 7.2000e-01 5.5000e-01 7.7000e-01]
 [1.7140e+01 4.4290e+01 5.2900e+00 3.9000e-01 3.2400e+00 8.3800e+00]
 [1.5000e+01 3.7500e+01 2.3500e+01 4.0000e-01 6.4000e-01 1.6000e+00]
 [5.0000e+01 5.5000e+01 1.2150e+03 9.1000e-01 4.0000e-02 5.0000e-02]
 [4.5000e+01 4.0000e+01 4.0075e+02 1.1200e+00 1.1000e-01 1.0000e-01]]
1
[[19.38 50.62 54.12  0.38  0.36  0.94]
 [10.   37.5   4.5   0.27  2.22  8.33]
 [14.   40.    1.6   0.35  8.75 25.  ]
 [15.   37.5  23.5   0.4   0.64  1.6 ]
 [24.38 33.75  4.88  0.72  5.    6.92]
 [11.67

[[3.0000e+01 5.1670e+01 3.1033e+02 5.8000e-01 1.0000e-01 1.7000e-01]
 [1.8330e+01 4.1670e+01 1.2000e+01 4.4000e-01 1.5300e+00 3.4700e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [1.0000e+01 3.7500e+01 4.5000e+00 2.7000e-01 2.2200e+00 8.3300e+00]
 [2.1670e+01 3.0000e+01 3.9170e+01 7.2000e-01 5.5000e-01 7.7000e-01]]
2
[[1.0000e+01 3.7500e+01 4.5000e+00 2.7000e-01 2.2200e+00 8.3300e+00]
 [2.1670e+01 3.0000e+01 3.9170e+01 7.2000e-01 5.5000e-01 7.7000e-01]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [1.6250e+01 6.3750e+01 1.4500e+01 2.5000e-01 1.1200e+00 4.4000e+00]
 [4.5000e+01 4.0000e+01 4.0075e+02 1.1200e+00 1.1000e-01 1.0000e-01]]
1
[[1.9380e+01 5.0620e+01 5.4120e+01 3.8000e-01 3.6000e-01 9.4000e-01]
 [1.0000e+01 3.7500e+01 4.5000e+00 2.7000e-01 2.2200e+00 8.3300e+00]
 [1.4620e+01 7.1920e+01 6.00